# Budowanie z modelami rodziny Meta

## Wprowadzenie

W tej lekcji omówimy:

- Poznanie dwóch głównych modeli z rodziny Meta – Llama 3.1 i Llama 3.2
- Zrozumienie zastosowań i scenariuszy dla każdego z modeli
- Przykład kodu pokazujący unikalne cechy każdego modelu

## Rodzina modeli Meta

W tej lekcji przyjrzymy się dwóm modelom z rodziny Meta, czyli „Llama Herd” – Llama 3.1 i Llama 3.2

Modele te występują w różnych wariantach i są dostępne na rynku modeli Github. Więcej informacji o korzystaniu z modeli Github do [prototypowania z modelami AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Warianty modeli:
- Llama 3.1 – 70B Instruct
- Llama 3.1 – 405B Instruct
- Llama 3.2 – 11B Vision Instruct
- Llama 3.2 – 90B Vision Instruct

*Uwaga: Llama 3 jest również dostępny na Github Models, ale nie będzie omawiany w tej lekcji*


## Llama 3.1

Dzięki 405 miliardom parametrów, Llama 3.1 zalicza się do kategorii otwartoźródłowych modeli LLM.

Ten model to ulepszona wersja wcześniejszego Llama 3, oferująca:

- Większe okno kontekstu – 128 tys. tokenów zamiast 8 tys.
- Większa maksymalna liczba tokenów wyjściowych – 4096 zamiast 2048
- Lepsze wsparcie dla wielu języków – dzięki zwiększonej liczbie tokenów treningowych

Te usprawnienia pozwalają Llama 3.1 obsługiwać bardziej złożone przypadki użycia podczas budowania aplikacji GenAI, w tym:
- Natywne wywoływanie funkcji – możliwość korzystania z zewnętrznych narzędzi i funkcji poza standardowym przepływem LLM
- Lepsza wydajność RAG – dzięki większemu oknu kontekstu
- Generowanie syntetycznych danych – możliwość tworzenia skutecznych danych do zadań takich jak dostrajanie modelu


### Wywoływanie funkcji natywnych

Llama 3.1 została dostrojona, aby skuteczniej wykonywać wywołania funkcji lub narzędzi. Model posiada także dwa wbudowane narzędzia, które potrafi samodzielnie rozpoznać jako potrzebne na podstawie polecenia użytkownika. Są to:

- **Brave Search** – Może być używane do uzyskiwania aktualnych informacji, takich jak pogoda, poprzez wyszukiwanie w internecie
- **Wolfram Alpha** – Może służyć do bardziej złożonych obliczeń matematycznych, dzięki czemu nie trzeba pisać własnych funkcji.

Możesz również tworzyć własne, niestandardowe narzędzia, które LLM będzie mógł wywoływać.

W poniższym przykładzie kodu:

- Definiujemy dostępne narzędzia (brave_search, wolfram_alpha) w poleceniu systemowym.
- Wysyłamy polecenie użytkownika z pytaniem o pogodę w konkretnym mieście.
- LLM odpowie wywołaniem narzędzia Brave Search, które będzie wyglądać tak: `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Uwaga: Ten przykład wykonuje jedynie wywołanie narzędzia. Jeśli chcesz uzyskać wyniki, musisz założyć darmowe konto na stronie Brave API i samodzielnie zdefiniować funkcję.*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Mimo że Llama 3.1 jest dużym modelem językowym, jej ograniczeniem jest brak obsługi multimodalności. Chodzi o możliwość korzystania z różnych typów danych wejściowych, takich jak obrazy, jako podpowiedzi i generowanie odpowiedzi. Ta funkcja jest jednym z głównych atutów Llama 3.2. Do nowych możliwości należą także:

- Multimodalność – potrafi analizować zarówno tekst, jak i obrazy jako podpowiedzi
- Wersje o małych i średnich rozmiarach (11B i 90B) – zapewniają elastyczne opcje wdrożenia,
- Wersje tylko tekstowe (1B i 3B) – pozwalają na wdrożenie modelu na urządzeniach brzegowych lub mobilnych i gwarantują niskie opóźnienia

Obsługa multimodalności to duży krok naprzód w świecie otwartych modeli. Przykład kodu poniżej pokazuje, jak można użyć zarówno obrazu, jak i tekstu jako podpowiedzi, aby uzyskać analizę obrazu z Llama 3.2 90B.

### Obsługa multimodalności w Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Nauka nie kończy się tutaj, kontynuuj swoją podróż

Po ukończeniu tej lekcji zajrzyj do naszej [kolekcji materiałów o Generative AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst), aby dalej rozwijać swoją wiedzę na temat Generative AI!



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony przy użyciu usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było poprawne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być traktowany jako źródło nadrzędne. W przypadku informacji o kluczowym znaczeniu zalecamy skorzystanie z profesjonalnych usług tłumaczeniowych. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
